In [1]:
%cd ..

/home/ubuntu/dev/reid/pepper


In [19]:
from copy import deepcopy

import torch
import numpy as np

import mmcv
from mmcv import Config

In [3]:
# %load_ext autoreload
# %autoreload 2

In [4]:
from pepper.datasets import build_dataset, build_dataloader, build_sampler

In [27]:
# load dataset configuration file

cfg_file = "configs/image/_base_/datasets/market1501.py"
# cfg_file = "tests/configs/_base_/datasets/mini_market1501.py"

cfg = Config.fromfile(cfg_file)
print(cfg.pretty_text)

img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True),
    dict(type='Resize', size=(256, 128), interpolation='bilinear'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Collect', keys=['img', 'gt_label']),
    dict(type='FormatBundle')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(256, 128), interpolation='bilinear'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'], meta_keys=[])
]
data_type = 'Market1501Dataset'
data_root = 'data/market1501/Market-1501-v15.09.15/'
data = dict(
    samples_per_gp

In [28]:
# get train datast
dataset = build_dataset(cfg.data.train)

print("dataset size:", len(dataset))
print("# of pids:", dataset.NUM_PIDS)
print("# of camids:", dataset.NUM_CAMIDS)

dataset size: 12936
# of pids: 751
# of camids: 6


In [29]:
batch_size = 32
num_instances = 4

shuffle = False

default_args = dict(
    dataset=dataset,
    num_replicas=2,
    rank=0,
)

sampler_cfg = dict(
    type="InfiniteBalancedIdentityDistributedSampler",
    # type="InfiniteNaiveIdentityDistributedSampler",
    batch_size=batch_size,
    num_instances=num_instances,
    shuffle=shuffle,
    round_up=True,
    seed=0,
)

sampler = build_sampler(sampler_cfg, default_args=default_args)

data_info = deepcopy([info["img_info"] for info in dataset.data_infos])

In [30]:
iter_samples = iter(sampler)

for i in range(batch_size * 2):
    
    idx = next(iter_samples)
    
    info = data_info[idx]
    
    print("{}\t{}\t{}".format(i % batch_size, info['pid'], info['camid']))

0	474	5
1	474	4
2	474	2
3	474	4
4	113	4
5	113	5
6	113	1
7	113	5
8	266	1
9	266	2
10	266	5
11	266	5
12	31	3
13	31	0
14	31	0
15	31	0
16	549	2
17	549	1
18	549	5
19	549	1
20	533	0
21	533	2
22	533	1
23	533	5
24	721	0
25	721	5
26	721	1
27	721	2
28	14	5
29	14	2
30	14	2
31	14	2
0	648	4
1	648	1
2	648	2
3	648	1
4	175	5
5	175	1
6	175	1
7	175	4
8	122	5
9	122	4
10	122	4
11	122	2
12	162	5
13	162	4
14	162	1
15	162	1
16	386	5
17	386	2
18	386	0
19	386	4
20	271	0
21	271	3
22	271	3
23	271	3
24	530	2
25	530	0
26	530	5
27	530	1
28	457	1
29	457	0
30	457	5
31	457	2


In [10]:
indices = []

for s in iter_samples:
    indices.append(s)

KeyboardInterrupt: 

In [11]:
len(indices)

796480

In [90]:
(12936 / 32) / 2

202.125

In [12]:
len(indices) / 32

47.0

In [93]:
12936 / 64

202.125